## load prediction results

In [ ]:
import numpy as np

all_y_test = np.load("Data/Predicted_SIC.npy")
all_y_pred = np.load("Data/Grounded_SIC.npy")

In [ ]:
# binary the sic based on the 0.15 threshold

all_y_test_binary = all_y_test[all_y_test<0.15] = 0
all_y_test_binary = all_y_test[all_y_test>=0.15] = 1

all_y_pred_binary = all_y_pred[all_y_pred<0.15] = 0
all_y_pred_binary = all_y_pred[all_y_pred>=0.15] = 1

In [21]:
all_y_test = np.reshape(all_y_test, (dim0,dim1,dim2,dim3,dim4,dim5))
all_y_pred = np.reshape(all_y_pred, (dim0,dim1,dim2,dim3,dim4,dim5))

In [22]:
# calculate the sie
all_y_test_SIE = np.sum(all_y_pred_binary, axis = (-2,-1)) # the axis depends on your array shape
all_y_pred_SIE = np.sum(all_y_pred_binary, axis = (-2,-1)) # the axis depends on your array shape

## calculate mean and anomaly

In [38]:
y_test_SIE_mean = np.mean(all_y_test_SIE, axis = 1)
y_pred_SIE_mean = np.mean(all_y_pred_SIE, axis = 1)

In [39]:
y_test_SIE_mean = np.expand_dims(y_test_SIE_mean, axis = 0)
y_pred_SIE_mean = np.expand_dims(y_pred_SIE_mean, axis = 0)

In [41]:
y_test_SIE_anom = all_y_test_SIE - y_test_SIE_mean
y_pred_SIE_anom = all_y_pred_SIE - y_pred_SIE_mean

## detrend

In [45]:
from scipy import signal

x = y_pred_SIE_anom.shape[1]
y = y_pred_SIE_anom.shape[2]
z = y_pred_SIE_anom.shape[3]

y_pred_SIE_anom_dtrend = signal.detrend(all_y_pred_SIE, axis=1, type='linear', bp=0, overwrite_data=False)
y_test_SIE_anom_dtrend = signal.detrend(all_y_test_SIE, axis=1, type='linear', bp=0, overwrite_data=False)


In [46]:
y_pred_SIE_anom = y_pred_SIE_anom_dtrend
y_test_SIE_anom = y_test_SIE_anom_dtrend

In [48]:
# activate regions for calculating BACC
activate_regions = np.load("Data/Activate_regions_monthly.npy")

# calculating the BACC
def Calculate_BACC(y_ground, y_pred):
    
    year_numb = y_ground.shape[0]
    month_ver = y_ground.shape[1]
    leadt_hor = y_ground.shape[4]
    
    all_bacc_matrix = np.zeros((year_numb, month_ver,leadt_hor),dtype=np.float64)
    
    cur_predit_sic = copy.deepcopy(y_pred)
    cur_ground_sic = copy.deepcopy(y_ground)
    
    cur_predit_sic[cur_predit_sic>0.15] = 1
    cur_ground_sic[cur_ground_sic>0.15] = 1
    
    cur_predit_sic[cur_predit_sic<=0.15] = 0
    cur_ground_sic[cur_ground_sic<=0.15] = 0
   
    for y in range(year_numb):

        for v in range(month_ver):
        
            for i in range(leadt_hor): 
                
                cur_pred_sie = cur_predit_sic[y,v,:,:,i]
                cur_true_sie = cur_ground_sic[y,v,:,:,i]
                
                cur_iiee = np.sum(np.abs(cur_pred_sie-cur_true_sie))
                cur_bacc = 1-cur_iiee/activate_regions[v]
                
                all_bacc_matrix [y,v,i] = cur_bacc
                
    return all_bacc_matrix

def calculate_acc_SIE(ground_anomly_self, predict_anomly_self):
    
    all_person_cf = []
    all_person_cf_p = []
    all_person_cf_p_self = []
    
    row_n = ground_anomly_self.shape[-2]
    col_n = ground_anomly_self.shape[-1]
    
    cur_per = np.zeros((row_n,col_n),dtype=np.float64)
   
    for j in range(row_n):
            for k in range(col_n):
                cur_ground_self = ground_anomly_self[:,j,k]
                cur_predict_self = predict_anomly_self[:,j,k]
                    
                cur_ground_self = np.asarray(cur_ground_self).flatten()
                cur_predict_self = np.asarray(cur_predict_self).flatten()
                    
                person_self = stats.pearsonr(cur_predict_self, cur_ground_self)
                cur_per[j,k] = person_self[0]
                
    return  np.array(cur_per)

In [ ]:
# calculate metrics

cur_bacc_matrix = Calculate_BACC(all_y_test, all_y_pred)
cur_acc_sie_i = calculate_acc_SIE(y_test_SIE_anom, y_pred_SIE_anom)

## draw the results

In [ ]:
import seaborn as sns
import pandas as pd
columns_lst = range(1,y_length+1)
row_index = range(1,biweek_per_year+1)
cur_acc_sie_i = np.round(cur_acc_sie_i*100, 2)
df = pd.DataFrame(data = cur_acc_sie_i, index = row_index, columns = columns_lst)
#fig, ax_arr = plt.subplots(1,1, figsize=(4,8.5))
sns.set_context({"figure.figsize":(8,8)})
ax = sns.heatmap(df, vmin = 0, vmax = 80, cmap = 'RdBu_r', annot=True, fmt=".2f", annot_kws={"size":13})

In [ ]:
import seaborn as sns
import pandas as pd

columns_lst = range(1,y_length+1)
row_index = range(1,biweek_per_year+1)
cur_bacc_matrix = np.round(cur_bacc_matrix*100, 2)
df = pd.DataFrame(data = cur_bacc_matrix, index = row_index, columns = columns_lst)
sns.set_context({"figure.figsize":(8,8)})
ax = sns.heatmap(df, vmin = 80, vmax = 100, cmap = 'RdBu_r', annot=True, fmt=".2f", annot_kws={"size":13})